In [163]:
import pandas as pd
from pymongo import MongoClient
import json
import matplotlib.pyplot as plt

In [164]:
client = MongoClient('mongodb://localhost:27017')
db = client.artblocks
mfer = db.art

In [165]:
def parse(data):
    record = {}
    asset = data.get('asset')

    if asset == None:
        return None
    
    record['collection_slug'] = asset['collection']['slug']
    record['collection_name'] = asset['collection']['name']

    record['asset_id'] = asset['id']
    record['asset_name'] = asset['name']
    record['asset_contract_date'] = asset['asset_contract']['created_date']

    # Owner Info
    if asset['owner']['user'] != None:
        record['owner_username'] = asset['owner']['user']['username']
    record['owner_address'] = asset['owner']['address']

    # Transaction Info
    record['event_timestamp'] = data['event_timestamp']
    record['event_type'] = data['event_type']
    
    return record


In [166]:
result = list()

for i in mfer.find():
    if 'asset_events' in i.keys():
        for data in i['asset_events']:
            clean_data = parse(data)

            if clean_data != None:
                result.append(clean_data)
    else:
        None

In [167]:
df = pd.DataFrame(result)
df.to_csv('art.csv')